In [ ]:
elastic_rods_dir = '../elastic_rods/python/'
weaving_dir = './'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, webapp_compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(webapp_compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from webapp_compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern, write_per_ribbon_svg)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, write_distance_to_linkage_mesh, get_max_distance_to_target_surface, write_crossing_ribbon_info, write_crossing_ribbon_info_json)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis, render_video, combine_three_stage_video)
import json

In [ ]:
with open(osp.join(weaving_dir + 'woven_model.json')) as f:
    data = json.load(f)

In [ ]:
benchmark_data = []
data_root = 'mega_monster_optimization_diagram_results/tenth_round/'
output_root = '../../curwe_webapp/build/models/'

### Expoort Web app data

In [ ]:
def write_crossing_ribbon_info_json(linkage, filename, scale):
    ''' return the list of two ribbon indices per crossing and the list of crossing lists per ribbon
    '''
    ribbons = order_segments_by_ribbons(linkage)
    _, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, linkage, rest = True)
    all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

    def get_crossing_json(pos, ribbon_pair):
        return {'position': list(pos), 'ribbons': ribbon_pair}

    def get_ribbon_json(crossing_index_list):
        return {'crossings': list(crossing_index_list)}

    def get_ribbon_crossing_list(index):
        selected_list = []
        selected_ribbon = []
        for ribbon_index, index_list in enumerate(all_joint_index_list):
            if index in set(index_list):
                selected_ribbon.append(ribbon_index)
                selected_list.append(index_list)

        return selected_ribbon
    pairs_of_ribbons_per_crossing = [get_ribbon_crossing_list(i) for i in range(linkage.numJoints())]
    crossing_positions = linkage.jointPositions().reshape((linkage.numJoints(), 3)) * scale

    crossing_info_list = [get_crossing_json(crossing_positions[c_index], pairs_of_ribbons_per_crossing[c_index]) for c_index in range(len(pairs_of_ribbons_per_crossing))]

    ribbon_info_list = [get_ribbon_json(all_joint_index_list[index]) for index in range(len(all_joint_index_list))]

    crossing_ribbon_info = {'crossings': crossing_info_list, 'ribbons': ribbon_info_list}

    with open(filename, 'w') as f:
        json.dump(crossing_ribbon_info, f, indent=4)
    return crossing_ribbon_info


In [ ]:
import pickle, gzip
benchmark_data = []
for model_info in data['models']:
    thickness, width, name, use_constant_width, width_scale, scale_joint_weight, update_attraction_weight, number_of_updates, fix_boundary, only_two_stage, num_joints, num_segments, num_ribbons, num_sim_vars, num_design_vars = model_info['thickness'], model_info['width'], model_info['name'], model_info['constant_cross_section'], model_info['cross_section_scale'], model_info['scale_joint_weight'], model_info['update_attraction_weight'], model_info['number_of_updates'], model_info['fix_boundary'], model_info['only_two_stage'], model_info['num_joints'], model_info['num_segments'], model_info['num_ribbons'], model_info['num_sim_vars'], model_info['num_design_vars']
    io = InputOrganizer(name, thickness, width, weaving_dir)
    stage_2_pickle_name = os.path.join(data_root, '{}/{}_stage_2.pkl.gz'.format(name, name))
    stage_3_pickle_name = os.path.join(data_root, '{}/{}_stage_3.pkl.gz'.format(name, name))
    ribbon_folder = '{}/{}'.format(output_root, name)
    
    if not os.path.exists(ribbon_folder):
        os.makedirs(ribbon_folder)
    if only_two_stage:
        curved_linkage = pickle.load(gzip.open(stage_2_pickle_name, 'r'))
    else:
        curved_linkage = pickle.load(gzip.open(stage_3_pickle_name, 'r'))
    
    two_ribbon_per_crossing, crossing_list_per_ribbon = write_crossing_ribbon_info_json(curved_linkage, '{}/{}/model.json'.format(output_root, name, name), 100)
    save_path = os.getcwd()
    write_per_ribbon_svg(curved_linkage, min(io.RIBBON_CS), io.SUBDIVISION_RESOLUTION, ribbon_folder)
    os.chdir(save_path)

### Setup webapp file structure

In [ ]:
import pickle, gzip
model_path_data = []
for model_info in data['models']:
    name, display_name = model_info['name'], model_info['display_name']
    model_path_data.append({
      "label": display_name,
      "id": name
    })
with open(output_root + 'models.json', 'w') as f:
    json.dump({"models" : model_path_data}, f, indent=4)

In [ ]:
import pickle, gzip
from shutil import copyfile

benchmark_data = []
for model_info in data['models']:
    thickness, width, name, use_constant_width, width_scale, scale_joint_weight, update_attraction_weight, number_of_updates, fix_boundary, only_two_stage, num_joints, num_segments, num_ribbons, num_sim_vars, num_design_vars = model_info['thickness'], model_info['width'], model_info['name'], model_info['constant_cross_section'], model_info['cross_section_scale'], model_info['scale_joint_weight'], model_info['update_attraction_weight'], model_info['number_of_updates'], model_info['fix_boundary'], model_info['only_two_stage'], model_info['num_joints'], model_info['num_segments'], model_info['num_ribbons'], model_info['num_sim_vars'], model_info['num_design_vars']
    io = InputOrganizer(name, thickness, width, weaving_dir)
    if 'kitty_level' in name:
        continue
    copyfile(io.SURFACE_PATH[:-4] + '00.obj', '{}/{}/{}'.format(output_root, name, 'target_surface.obj'))
    copyfile(io.MODEL_PATH[:-4] + '00.obj', '{}/{}/{}'.format(output_root, name, 'topology.obj'))

In [ ]:
io.SURFACE_PATH, output_root + '/' + name

In [ ]:
io.MODEL_PATH

In [ ]:
import numpy as np 
from numpy import linalg as la

linkage_file = weaving_dir + "models/{}.obj"
surface_file = weaving_dir + 'surface_models/{}.obj'

output_linkage_file = weaving_dir + "normalized_objs/models/{}.obj"
output_surface_file = weaving_dir + 'normalized_objs/surface_models/{}.obj'

def scale_model(target_size, linkage_file, surface_file, output_linkage_file, output_surface_file):
    point_list = []
    edge_list = []
    face_list = []
    with open(linkage_file, 'r') as linkage, open(output_linkage_file, 'w') as output_linkage, open(surface_file, 'r') as surface, open(output_surface_file, 'w') as output_surface:
        content = linkage.readlines()
        count = 0
        edge_count = 0
        for line in content:
            if 'v ' in line:
                point = np.array([float(x) for x in line.split(' ')[1:]])
                point_list.append(point)

            if 'l ' in line:
                edge_list.append(line)

            if 'f ' in line:
                face_list.append(line)

        cm = np.sum(np.array(point_list), axis = 0) / len(point_list)

        centered_point_list = []
        point_norm = []
        for point in point_list:
            point = np.array(point) - cm
            centered_point_list.append(point)
            point_norm.append(la.norm(point))

        scale = target_size / max(point_norm)
        print(scale, cm)

        for line in content:
            if 'v ' in line:
                point = np.array([float(x) for x in line.split(' ')[1:]])
                point = scale * (point - cm)
                output_linkage.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
            else:
                output_linkage.write(line)

        surface_content = surface.readlines()
        for line in surface_content:
            if 'v ' in line:
                point = np.array([float(x) for x in line.split(' ')[1:]])
                point = scale * (point - cm)
                output_surface.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
            else:
                output_surface.write(line)

linkage_name_list = [model_info['name'] for model_info in data['models']]
surface_name_list = linkage_name_list
target_size = 100
for linkage_name, surface_name in zip(linkage_name_list, surface_name_list):
    scale_model(target_size, output_linkage_file.format(linkage_name), output_surface_file.format(surface_name), output_linkage_file.format(linkage_name+'_{}'.format(str(target_size))), output_surface_file.format(surface_name+'_{}'.format(str(target_size))))

In [ ]:
[model_info['name'] for model_info in data['models']]

In [ ]:
import pickle, gzip
model_path_data = []
for model_info in data['models']:
    name, display_name = model_info['name'], model_info['display_name']
    if not os.path.exists('{}/{}/ribbons'.format(output_root, name)):
        os.makedirs('{}/{}/ribbons'.format(output_root, name))